In [3]:
from pathlib import Path
from natsort import natsorted, ns
import cv2

import numpy as np
import io
import json
import requests
import glob

from tensorflow.keras.models import load_model

import sys
sys.path.append('/models/')
from preprocess import get_image, resize, preprocess_mobile_image, sentences_segmentate, resize_sentences_for_model, preprocess_for_model
from pathlib import Path
from natsort import natsorted, ns

In [6]:
img = get_image("../sc0002.jpg")

In [9]:
url_api = "https://api.ocr.space/parse/image"
_, compressedImage = cv2.imencode(".jpg", img, [1, 90]) #Free tier API only allows image <1MB
file_bytes = io.BytesIO(compressedImage)

result = requests.post(url_api, files = {"screenshot.jpg": file_bytes}, data = {"apikey": "ff0f1158f788957", "language": "eng"})
result = result.content.decode()
result = json.loads(result)
parsed_results = result.get("ParsedResults")[0]
text_detected = parsed_results.get("ParsedText")


In [12]:
print(text_detected)

text_list = text_detected.split('\n')
text_list_strip = [b.replace('\r', '') for b in text_list]


2 tablespoons cornstarch
1 Y2 tablespoons water
6 cups chicken broth
2 Y2 tablespoons soy sauce
1 tablespoon fish sauce
1 tablespoon rice vinegar



In [15]:
text_list_strip_1 = [b.replace('Y2', '1/2') for b in text_list_strip]
text_list_strip_2 = [b.replace('Y3', '1/3') for b in text_list_strip_1]
text_list_strip_3 = [b.replace('Y4', '1/4') for b in text_list_strip_2]
text_list_strip = [b.replace('Y8', '1/8') for b in text_list_strip_3]

text_list_strip = filter(None, text_list_strip)

In [25]:
import sys
sys.path.append('/models/')
import sklearn_crfsuite as crf
from models.parsing import tokenize, standardize, asfloat
from models.evaluate import getlabels
from models.training import removeiob, getfeatures

ModuleNotFoundError: No module named 'sklearn_crfsuite'

In [ ]:
crf_model = "models/model.crfsuite"

import requests
import json

def displaytags(tokens, tags):
    # Make a table with pandas and transpose to make horizontal
    df = pd.DataFrame(tokens, removeiob(pred)).transpose()
    # Print string representation with adjusted spacing and display options
    print(df.to_string(index=False, justify='center', col_space=8, max_cols=15))

INGREDIENTS = ["B-INGR", "I-INGR"]
QTY = ["B-QTY", "I-QTY"]
UNITS = ["B-UNIT", "I-UNIT"]
IMPERIALS = ["tablespoon", "teaspoon", "cup", "ounce", "pound"]

In [24]:
converted_grams = []

for item in text_list_strip:
    ingredient_list = []
    qty_list = []
    unit_list = []

    tokens = tokenize(item, preprocess=True)
    preds = getlabels(item, crf_model)
    print(tokens)
    print(preds)
    #displaytags(tokens, preds)

    for token, pred in zip(tokens, preds):
        if (pred in INGREDIENTS):
            ingredient_list.append(token)
        elif (pred in QTY):
            qty_list.append(token)
        elif (pred in UNITS):
            unit_list.append(token)

        ingredient_string = ' '.join(ingredient_list)
        qty_string = ' '.join(qty_list)
        qty_string = asfloat(qty_string)
        unit_string = ' '.join(unit_list)
        unit_string = standardize(unit_string)
        print(unit_string)
    #print('-'*100)
    
    if (unit_string in IMPERIALS):
        data = {
            "apiKey": "ef6e10dabdeb46d7a0088c6ea82b55ed",
            "ingredientName": ingredient_string,
            "sourceAmount": qty_string,
            "sourceUnit": unit_string,
            "targetUnit": "grams"
        }

        datajson = json.dumps(data)
        responselogin = requests.get(url='https://api.spoonacular.com/recipes/convert', params=data)
        #print(responselogin.status_code)
        result = responselogin.json()
        print(result)

        grams = result['answer']
        if(grams is not None):
            converted_grams = converted_grams.append(grams)
    print('-'*100)

print(converted_grams)


NameError: name 'tokenize' is not defined